# EDA
## Análisis Exploratorio de los Datos del archivo Homicidios.xlsx

In [199]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

Importo los datos contenidos en el archivo homicidios.xlsx, que trae dos hojas: HECHOS y VICTIMAS. Almaceno en dos dataframes independientes

In [200]:
df_HH = pd.read_excel('homicidios.xlsx', sheet_name= "HECHOS")
df_HV = pd.read_excel('homicidios.xlsx', sheet_name= "VICTIMAS")

### Revision dataframe de HECHOS (df_HH)

In [201]:
df_HH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

#### Revisión de Datos Nulos

In [202]:
df_HH.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

De esta primera exploración, sumada a una revisión manual al abrir el archivo en excel, se puede apreciar que es un dataset bastante limpio y casi listo para trabajar. Tiene información completa, es decir sin datos nulos, en las columnas ID, N_VICITMAS, FECHA, AAA, MM, DD, HORA, LUGAR DEL HECHO, TIPO DE CALLE, COMUNA, PARTICIPANTES, VICTIMA, ACUSADO.

EN LAS COLUMNAS XY (CABA), pos x, pos y, aparentemente no hay datos nulos, pero al revisar en excel se pueden encontrar varios registros en los cuales no hay realmente información en estos campos sino puntos (.), lo que para efectos prácticos equivale a valores nulos o vacíos.

Otras columnas con algunos datos nulos son las que corresponden al desglose de la dirección: Calle, Altura, Cruce, Dirección Normalizada.

Los nombres que traen las columnas tal vez nos daban ya esa información, pues las columnas que están completas tienen su nombre todo escrito en mayúsculas (Con excepción de XY CABA), y las incompletas están en minúsculas.

Un reto grande con el que me encuentro, al no ser ciudadana Argentina, es entender cómo funcionan las direcciones en la ciudad.

NOTA PAR MI:
- Revisar los ocho registros que no tienen información en 'Dirección Normalizada' para ver si puedo completarlos con la demás información que poseo. Esto tal vez haga parte de una revisión de las columnas 'LUGAR_DEL_HECHO' y 'Dirección Normalizada' pues aparentemente contienen la misma información escrita en formatos diferentes.

#### Columnas XY (CABA), pos x, pos y

La geolocalización es importante para los ánalisis que se deben realizar, así que no quiero eliminar los registros en los que estos no existen. Intentaré utilizar la información que poseo sobre el lugar del accidente para completar la que hace falta. Para esta tarea utilizaré GeoPy que es una librería de python para acceder a servicios de geocodificación.

Mi objetivo es conseguir los valores de las coordenadas X, Y. El valor de XY(CABA) -geocodificación plana-, podría ser útil a la hora de utilizar mapas pero eso lo decidiré más adelante.

In [203]:
#Observo los registros que necesito completar:
df_sinXY = df_homicidios[df_homicidios['XY (CABA)']=='Point (. .)'][['LUGAR_DEL_HECHO', 'Dirección Normalizada', 'XY (CABA)', 'pos x', 'pos y']]
df_sinXY

,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238
38,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.
71,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973
106,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.
119,SD,NaN,Point (. .),.,.
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.
176,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.
181,AU DELLEPIANE 2400,NaN,Point (. .),.,.
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.


Utilizo geolocator para conseguir las coordenadas geográficas a partir del LUGAR_DEL_HECHO

In [204]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=10)

def do_geocode(address):
     return geolocator.geocode(address)


In [206]:
df_sinXY['coordenadas'] =df_sinXY['LUGAR_DEL_HECHO'].apply(do_geocode).apply(lambda loc: tuple(loc.point) if loc else None)
df_sinXY

,LUGAR_DEL_HECHO,Dirección Normalizada,XY (CABA),pos x,pos y,coordenadas
35,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),-58.37714647568196,-34.63657525428238,None
38,AUTOPISTA LUGONES PK 10000,NaN,Point (. .),.,.,None
71,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,Point (. .),-58.47433193007387,-34.66684950051973,None
106,AU BUENOS AIRES - LA PLATA KM. 4,NaN,Point (. .),.,.,None
119,SD,NaN,Point (. .),.,.,"(14.5844444, 29.4917691, 0.0)"
139,AUTOPISTA 25 DE MAYO,AUTOPISTA 25 DE MAYO,Point (. .),.,.,"(-34.6425804, -58.461474, 0.0)"
176,AV. LEOPOLDO LUGONES PKM 6900,"LUGONES, LEOPOLDO AV.",Point (. .),.,.,None
180,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,NaN,Point (. .),.,.,None
181,AU DELLEPIANE 2400,NaN,Point (. .),.,.,"(-31.4271733, -64.2087771, 0.0)"
256,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,Point (. .),.,.,None


Necesito encontrar la forma de sacar de la columna 'coordenadas' los valores indivuduales de X e Y y asignarlos a las columnas correspondientes en el dataset original. Mientras consigo hacer esto, lo que voy a hacer es rellenar estos espacios vacíos con ceros, para efectos del dashboard

In [208]:
df_HH['pos x'] = df_HH['pos x'].replace('.', 0)
df_HH['pos y'] = df_HH['pos y'].replace('.', 0)
df_HH['XY (CABA)'] = df_HH['XY (CABA)'].replace('Point (. .)', 0)

#### Columnas 'Altura' y 'Cruce'

Son las dos columnas que presentan más valores nulos, y puedo ver que son complementarias en la información que contienen.
Indican si el accidente ocurrió en una esquina (cruce de vías), o en la mitad de una cuadra. Es una información importante que se puede resumir en una sola columna, en la que consignaré si el accidente ocurrió o no ocurrió en un cruce. Esto implicará conservar la columna 'Cruce' cambiando sus valores a True/False y eliminar la columna 'Altura'.
De esta manera también la Columna 'Cruce' quedaría completa, sin valores nulos.

In [209]:
df_HH = df_HH.drop('Altura', axis=1)

In [210]:
df_HH['Cruce'] = df_HH['Cruce'].notna()
df_HH.head(15)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,27 DE FEBRERO AV.,True,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
6,2016-0009,1,2016-01-24,2016,1,24,19:10:00,19,NOGOYA Y JOAQUIN V. GONZALES,CALLE,NOGOYA,True,"NOGOYA y GONZALEZ, JOAQUIN V.",11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,MOTO-AUTO,MOTO,AUTO
7,2016-0010,1,2016-01-29,2016,1,29,15:20:00,15,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,MOTO-AUTO,MOTO,AUTO
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,BELGRANO AV.,True,"BELGRANO AV. e IRIGOYEN, BERNARDO DE",1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,MOTO-CARGAS,MOTO,CARGAS
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,ENTRE RIOS AV.,False,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO


#### Columna ID y revisión de duplicados

No necesita ningún cambio, únicamente validación del tipo de dato que debe ser string.
Uso esta columna para verificar si hay duplicados. La revisión me muestra que no los hay.

In [211]:
df_HH['ID'].duplicated().sum()

0

#### Columna N_VICTIMAS
Esta columna contiene valores de tipo entero y no tiene valores nulos. Está lista para ser utilizada.

#### Columnas FECHA, AAAA, MM, DD, HORA Y HH

Estas columnas contienen información que puede ser redundante, pero no me estorba, no tienen valores nulos que puedan entorpecer las consultas y por lo tanto decido conservarlas como están. Sólo falta verificar tipos de datos.

#### Columnas LUGAR_DEL_HECHO y Direccion Normalizada
Ambas columnas almacenan la misma información, sólo cambia el formato. En 'Dirección Normalizada' Hay algunos valores nulos.

In [212]:
df_HH[df_HH['Dirección Normalizada'].isnull()][['Dirección Normalizada', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE']]

,Dirección Normalizada,LUGAR_DEL_HECHO,TIPO_DE_CALLE
38,NaN,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,NaN,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,NaN,SD,CALLE
180,NaN,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,NaN,AU DELLEPIANE 2400,AUTOPISTA
313,NaN,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,NaN,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,NaN,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


Son pocos datos pero no tengo información para construir esa dirección con el formato requerido, así que le asigno el valor SD para conservar estos registros

In [213]:
# Se reemplazan los valores nulos con SD
df_HH['Dirección Normalizada'].fillna("SD", inplace=True)
# Se verifican los datos imputados
df_HH[df_HH['Dirección Normalizada']== 'SD'][['Dirección Normalizada', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE']]

,Dirección Normalizada,LUGAR_DEL_HECHO,TIPO_DE_CALLE
38,SD,AUTOPISTA LUGONES PK 10000,AUTOPISTA
106,SD,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA
119,SD,SD,CALLE
180,SD,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA
181,SD,AU DELLEPIANE 2400,AUTOPISTA
313,SD,AUTOPISTA LUGONES KM 4.7,AUTOPISTA
546,SD,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA
621,SD,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA


#### Columna Calle
Esta columna tiene un sólo valor nulo. Reviso si puedo corregirlo manualmente con la información de los otros campos

In [214]:
df_HH[df_HH['Calle'].isna()]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,NaN,False,SD,0,0,0,0,PEATON-SD,PEATON,SD


Es un campo que contiene muy poca información. Nada sobre el lugar de accidente ni sobre el acusado. Considero que todos los registros suman en las estadísticas de temas tan delicados como este, así que decido conservar el registro, colocando SD en los campos que está vacíos.

In [215]:
df_HH['Calle'][df_HH['ID']=='2016-0151'] = 'SD'

Hago un nuevo conteo de valores nulos para verificar que no haya quedado ninguno

In [216]:
df_HH.isnull().sum()

ID                       0
N_VICTIMAS               0
FECHA                    0
AAAA                     0
MM                       0
DD                       0
HORA                     0
HH                       0
LUGAR_DEL_HECHO          0
TIPO_DE_CALLE            0
Calle                    0
Cruce                    0
Dirección Normalizada    0
COMUNA                   0
XY (CABA)                0
pos x                    0
pos y                    0
PARTICIPANTES            0
VICTIMA                  0
ACUSADO                  0
dtype: int64

#### Columnas TIPO_DE_CALLE, COMUNA, PARTICIPANTES

Estas columnas no tienen valores nulos. Están listas para ser utilizadas.

#### Columnas VICTIMA y ACUSADO
Reviso si las categorías presentes en estas columnas corresponden a las informadas en el Diccionario de Datos

In [223]:
df_HH['ACUSADO'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

Encuentro un valor que no corresponde al los del dccionario, 'MULTIPLE', voy a revisar los registros que lo contienen.

In [224]:
df_HH[df_HH['ACUSADO'].isin(['MULTIPLE'])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
31,2016-0044,1,2016-03-31,2016,3,31,19:40:00,19,AV JOSE MARIA MORENO Y BALBASTRO,AVENIDA,"MORENO, JOSE MARIA AV.",True,"MORENO, JOSE MARIA AV. y BALBASTRO",7,Point (102857.73817324 99148.76714504),-58.43213235,-34.63693953,MULTIPLE,MOTO,MULTIPLE
143,2016-0178,1,2016-12-30,2016,12,30,16:18:00,16,REPUBLICA BOLIVARIANA DE VENEZUELA Y SAAVEDRA,CALLE,REPUBLICA BOLIVARIANA DE VENEZUELA,True,REPUBLICA BOLIVARIANA DE VENEZUELA y SAAVEDRA,3,Point (105541.07964363 101532.48653010),-58.40288226,-34.61544091,MULTIPLE,PEATON,MULTIPLE
146,2017-0007,1,2017-01-11,2017,1,11,17:00:00,17,LIMA Y MEXICO,CALLE,LIMA,True,LIMA y MEXICO,1,Point (107491.43483173 101547.26343171),-58.38161660,-34.61529543,MULTIPLE,PEATON,MULTIPLE
153,2017-0014,1,2017-01-28,2017,1,28,08:27:34,8,AV. DONADO Y ARIAS,AVENIDA,DONADO,True,DONADO y ARIAS,12,Point (97378.59173196 108741.41437861),-58.49185972,-34.55046754,MULTIPLE,MOTO,MULTIPLE
174,2017-0040,1,2017-04-03,2017,4,3,09:40:00,9,CAMINO CIUDAD UNUVERSITARIA Y AV. TTE. GUIRALDES,AVENIDA,"GUIRALDES, INT.",False,"GUIRALDES, INT. 2699",13,Point (102290.82648731 109281.08885461),-58.43834197,-34.54560369,MULTIPLE,AUTO,MULTIPLE
225,2017-0101,1,2017-08-23,2017,8,23,10:00:00,10,AV. FCO. LACROZE Y ZAPATA,AVENIDA,"LACROZE, FEDERICO AV.",True,"LACROZE, FEDERICO AV. y ZAPATA",13,Point (101644.77699748 106480.78805418),-58.44537508,-34.57084818,MULTIPLE,PEATON,MULTIPLE
277,2018-0003,1,2018-01-05,2018,1,5,10:12:00,10,"Pueyrredon, Honorio, Dr. Av. y Galicia",AVENIDA,"PUEYRREDON, HONORIO, DR. AV.",True,"PUEYRREDON, HONORIO, DR. AV. y GALICIA",15,Point (101348.13972450 102808.02053976),-58.44860217,-34.60395657,MULTIPLE,PEATON,MULTIPLE
278,2018-0004,1,2018-01-09,2018,1,9,09:40:00,9,Del Libertador Av. y Manzanares,AVENIDA,DEL LIBERTADOR AV.,True,DEL LIBERTADOR AV. y MANZANARES,13,Point (100183.96103971 109533.84269558),-58.46129513,-34.54332766,MULTIPLE,MOTO,MULTIPLE
355,2018-0081,1,2018-07-29,2018,7,29,06:30:00,6,Castañares Av. y Rivera Indarte Av.,AVENIDA,CASTANARES AV.,True,CASTAÃ‘ARES AV. y RIVERA INDARTE AV.,7,Point (101453.89687005 98180.39027169),-58.44744156,-34.64567171,MULTIPLE,PEATON,MULTIPLE
375,2018-0101,1,2018-09-21,2018,9,21,00:15:00,0,"Paz, Gral. Av. y Zapiola",GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y ZAPIOLA",12,Point (98376.44189669 109821.71138325),-58.48098594,-34.54073130,MULTIPLE,AUTO,MULTIPLE


Veo que en todos estos accidentes los participantes fueron múltiples y hubo una sola vícitima fatal. Por ahora dejo esto sin realizar ninguna modificación.

In [217]:
df_HH['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

Encuentro dos valores que no corresponden, OBJETO FIJO y PEATON_MOTO, entonces voy a revisar los registros que los contienen

In [218]:
df_HH[df_HH['VICTIMA'].isin(['OBJETO FIJO', 'PEATON_MOTO'])]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


La información es confusa, pero revisando la hoja de Vícitimas para este mismo caso, se puede ver que en ambos accidentes hubo dos víctimas fatales. 
En el primer caso no tiene mucho sentido que la víctima haya sido un OBJETO FIJO, e cierto que el accidente fue una colisión contra un objeto fijo pero las víctimas fueron PASAJERO_ACOMPAÑANTE.
En el segundo caso, las vícitmas fatales fueron un PEATON y un PASAJERO_ACOMPAÑANTE.

Voy a revisar otros casos que hayan tenido 2 víctimas, para ver cómo están registrados en esta columna.

In [222]:
df_HH[df_HH['N_VICTIMAS']==2]

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
29,2016-0041,2,2016-03-29,2016,3,29,11:00:00,11,AV DIRECTORIO Y RIVERA INDARTE,AVENIDA,DIRECTORIO AV.,True,DIRECTORIO AV. y RIVERA INDARTE AV.,7,Point (100232.38564985 99530.25843190),-58.46076550,-34.63350444,MOTO-CARGAS,MOTO,CARGAS
97,2016-0126,2,2016-09-18,2016,9,18,22:45:00,22,IRIGOYEN Y TINOGASTA,CALLE,IRIGOYEN,True,IRIGOYEN y TINOGASTA,10,Point (94275.54271123 100886.87954649),-58.52572109,-34.62125906,AUTO-CARGAS,AUTO,CARGAS
161,2017-0026,2,2017-02-26,2017,2,26,05:15:00,5,AV. PERITO MORENO Y FOURNIER,AVENIDA,"MORENO, PERITO AV.",True,"MORENO, PERITO AV. y FOURNIER",4,Point (104113.67806500 97722.68219304),-58.41842777,-34.64979057,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
171,2017-0036,2,2017-03-29,2017,3,29,18:00:00,18,CURUPAYTI Y COLECTORA AV. GRAL. PAZ,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y CAMPANA",12,Point (95300.18060161 105287.23850698),-58.51452347,-34.58159762,MOTO-PASAJEROS,MOTO,PASAJEROS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,0,0,0,MOTO-CARGAS,MOTO,CARGAS
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO
234,2017-0112,2,2017-09-10,2017,9,10,09:32:17,9,AU 25 DE MAYO Y AV. VARELA,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO y VARELA AV.,7,Point (100479.14412395 98812.52258620),-58.45807401,-34.63997430,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
244,2017-0126,2,2017-10-14,2017,10,14,06:15:00,6,AV. RIESTRA Y CAÑADA DE GOMEZ,AVENIDA,RIESTRA AV.,True,RIESTRA AV. y CAÃ‘ADA DE GOMEZ,8,Point (98466.20276785 94207.42419155),-58.48003734,-34.68148508,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO
267,2017-0153,2,2017-12-11,2017,12,11,02:00:00,2,ROSARIO Y AV. LA PLATA,AVENIDA,ROSARIO,True,ROSARIO y LA PLATA AV.,6,Point (103186.65733155 101217.09689755),-58.42855267,-34.61829385,MOTO-PASAJEROS,MOTO,PASAJEROS
289,2018-0015,2,2018-02-06,2018,2,6,01:00:00,1,Australia Av. 2800,AVENIDA,AUSTRALIA AV.,False,AUSTRALIA AV. 2800,4,Point (107053.47849253 98047.93724144),-58.38636298,-34.64684275,MOTO-PASAJEROS,MOTO,PASAJEROS


Encuentro que no hay información sobre dos víctimas en la columna VICTIMA. Por ahora dejaré esto sin realizar modificación y lo atenderé de nuevo luego de realizar el análisis al dataframe df_HV, correspondiente a la hoja VICTIMAS  extraída del documento de excel.

NOTA PARA MI: Hasta aquí llego hoy viernes. Mañana intento extraer las coordenadas, pero no le voy a dedicar más de media hora. Luego Verifico tipos de datos, organizo los nombres de las columnas y paso a revisar el df_HV.

Cuando el df_HV esté revisado hago merge de ambos dataframes y organizo el dataframe resultante. Todo esto debería estar listo a medio día para pasar a trabajar los KPI.

El domingo trabajar sobre el dash board y el lunes sobre la presentación, el Reademe y el StoryTelling.